# Exploration of key words occurances in thread names notebook

In [1]:
import os
import sys
from src import data_prepare

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
post, thread=data_prepare.load_train_data()
label_map=data_prepare.load_label_map()
thread.head()

,thread_num,thread_name,thread_label,thread_replies,thread_label_id
0,45016,Games Threshold Discussion and List [Vote for ...,other,5703,8
1,88720,New Player's Guide to the Subforum - New to Ma...,other,961,8
2,39338,Mafia: A Basic Tutorial,other,79,8
3,34959,Paranormal Mafia Game - Rules Discussion,other,1719,8
4,64229,Notable Games Archive,other,307,8


In [3]:
key_words=pd.Series(label_map.index)
key_words

0             bastard
1     beginners-mafia
2                byor
3             classic
4        closed-setup
5              cybrid
6                kotm
7      non-mafia-game
8               other
9          paranormal
10       supernatural
11            vanilla
12           vengeful
Name: type_name, dtype: object

In [4]:
key_words[1]="beginner"
key_words[6]="king"
thread["name"]=thread["thread_name"].apply(lambda x: data_prepare.clean(x)).as_matrix()

 #### Let's just try to calculate how efficient it is to look for names themselves in thread names for different classes

In [5]:
num_classes=len(key_words)
correct=pd.Series(np.zeros(num_classes),index=label_map.index)
over=pd.Series(np.zeros(num_classes),index=label_map.index)
under=pd.Series(np.zeros(num_classes),index=label_map.index)
al=pd.Series(np.zeros(num_classes),index=label_map.index)

In [6]:
for item in thread.itertuples():
    label=item.thread_label_id
    if key_words[label] in item.name:
        correct[label]+=1
    else:
        under[label]+=1
    for index,s in enumerate(key_words):
        if index==label:
            continue
        if s in item.name:
            over[index]+=1
            break
    al[label]+=1

In [7]:
x = pd.DataFrame({'all':al,'correct':correct,'under': under,'over':over,
                  'per cent correct':correct/al,'per cent under':under/al,
                 'per cent over':over/al},
                 index=correct.index)
x

,all,correct,under,over,per cent correct,per cent under,per cent over
type_name,,,,,,,
bastard,14.0,7.0,7.0,4.0,0.500000,0.500000,0.285714
beginners-mafia,23.0,21.0,2.0,13.0,0.913043,0.086957,0.565217
byor,13.0,8.0,5.0,4.0,0.615385,0.384615,0.307692
classic,21.0,0.0,21.0,1.0,0.000000,1.000000,0.047619
closed-setup,36.0,0.0,36.0,0.0,0.000000,1.000000,0.000000
cybrid,3.0,3.0,0.0,1.0,1.000000,0.000000,0.333333
kotm,2.0,2.0,0.0,16.0,1.000000,0.000000,8.000000
non-mafia-game,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000
other,201.0,6.0,195.0,1.0,0.029851,0.970149,0.004975


In [8]:
x["efficiency"]=(x["correct"]-x["over"])/x['all']

In [9]:
x

,all,correct,under,over,per cent correct,per cent under,per cent over,efficiency
type_name,,,,,,,,
bastard,14.0,7.0,7.0,4.0,0.500000,0.500000,0.285714,0.214286
beginners-mafia,23.0,21.0,2.0,13.0,0.913043,0.086957,0.565217,0.347826
byor,13.0,8.0,5.0,4.0,0.615385,0.384615,0.307692,0.307692
classic,21.0,0.0,21.0,1.0,0.000000,1.000000,0.047619,-0.047619
closed-setup,36.0,0.0,36.0,0.0,0.000000,1.000000,0.000000,0.000000
cybrid,3.0,3.0,0.0,1.0,1.000000,0.000000,0.333333,0.666667
kotm,2.0,2.0,0.0,16.0,1.000000,0.000000,8.000000,-7.000000
non-mafia-game,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.000000
other,201.0,6.0,195.0,1.0,0.029851,0.970149,0.004975,0.024876


 ### Not the best metric for efficiency, but it seems like it makes sense to use this kind of raw prediction for classes that score more than 40-50%, especially if they don't have a lot of representatives in the dataset, like cybrid or supernatural

In [10]:
post_test, thread_test=data_prepare.load_test_data()

In [11]:
test_stat=pd.Series(np.zeros(13),index=label_map.index)
thread_test["name"]=thread_test["thread_name"].apply(lambda x: data_prepare.clean(x)).as_matrix()
for item in thread_test.itertuples():
    for index,s in enumerate(key_words):
        if s in item.name:
            test_stat[index]+=1

In [12]:
test_stat

type_name
bastard             8.0
beginners-mafia    35.0
byor               17.0
classic             0.0
closed-setup        0.0
cybrid              5.0
kotm                8.0
non-mafia-game      0.0
other               0.0
paranormal          7.0
supernatural        7.0
vanilla             2.0
vengeful            6.0
dtype: float64

### Thus, seemingly it does make sense to use key words prediction for:
- cybrid
- supernatural
- paranormal
- vengeful

#### Byor and beginners mafia would propably make sense too, if there weren't enough representatives of these classes, we may predict them well anyway(prediction statistics is included in another notebook), so we don't risk loosing precision overusing key words 